In [6]:
# Random sentence generation to test model hypothesis

In [15]:
import json
import datetime
import requests
import sys, traceback
import re
import os
import math
import random
from collections import Counter
import matplotlib.pyplot as plt
import csv

In [36]:
def mean(topic_no,total_topics,alphabet_length):
    return(topic_no*alphabet_length/(1.0*total_topics))

def make_sentence(alphabet_length,mean,std,sentence_len):
    sentence=[]
    for i in range(sentence_len):
        letter=int(round(random.gauss(mean,std),0))
        #if letter<0: letter+=alphabet_length
        #if letter>alphabet_length:letter-=alphabet_length
        sentence.append(letter)
    return sentence

In [12]:
alphabet_length=100
topics = 10
distros = []
for t in range(1,topics+1):
    distros.append((mean(t,topics,alphabet_length),random.uniform(.75,(1.75*alphabet_length)/(1.*topics))))
    
    
sent_lens = []
sents2=[]
for i in range(topics):
    sent = make_sentence(alphabet_length,distros[i][0],distros[i][1],100000)
    sent_lens.append(Counter(sent))
    sents2.append(sent)
    plt.plot(sent_lens[i].keys(),sent_lens[i].values(),'o')
    #plt.axis([-5, 15,0,27000])
max_word = max(map(lambda x: max(x),sents2))
min_word = min(map(lambda x: min(x),sents2))
plt.show()
print(max_word)
print(min_word)

146.0
-42.0


In [48]:
def doc_len_distro(x):
    return 5.991524457330881*x**2*math.exp(-0.00034375*x**2)

def generate_sentence_lens(sample_size):
    max_len = 200
    increment = 5
    samples = map(lambda x: (x,doc_len_distro(x)),range(0,205,5))
    s_samples = sum(zip(*samples)[1])
    scale = sample_size/s_samples
    return map(lambda x: (x[0],int(round(scale*x[1],0))),samples)

def generate_sentences(alphabet_len,mean,std,samples):
    sentences = []
    for ind in range(len(samples)):
        for sent_no in range(samples[ind][1]):
            sentence = make_sentence(alphabet_len,mean,std,samples[ind][0])
            sentences.append(sentence)
    maxi = max(map(lambda x: max(x),sentences))
    mini = min(map(lambda x: min(x),sentences))
    return mini,maxi,sentences 

def generate_corpus(alphabet_len,sample_size,topics):
    distros = []
    for t in range(1,topics+1):
        distros.append((
                mean(t,topics,alphabet_len),
                random.uniform(.75,(1.75*alphabet_len)/(1.*topics))
            ))
    
    samples = generate_sentence_lens(sample_size)
    corpus=[]
    global_mini = 0
    global_maxi = alphabet_len
    with open('corp.csv','a') as f:
        writer = csv.writer(f, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for me,std in distros:
            mini,maxi,sentences = generate_sentences(alphabet_len,me,std,samples)
            if mini<global_mini: global_mini=mini
            if maxi>global_maxi: global_maxi=maxi
            for sent in sentences:
                writer.writerow(sent)
    print('unique words: '+str(-global_mini+global_maxi))
    print('small_word: '+str(global_mini))
    return corpus

def rescale_corpus(filename,mini):
    with open(filename,'r') as f1:
        with open(filename.split('.')[0]+'scaled.csv','a') as f2:
            with open(filename.split('.')[0]+'natural.txt','a') as f3:
                writer = csv.writer(f2, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
                reader = csv.reader(f1,delimiter=',')
                for row in reader:
                    wr = map(lambda x: int(x)-mini,row)
                    writer.writerow(wr)
                    f3.write(' '.join([str(i)for i in wr])+'\n')
            
            

In [38]:
c = generate_corpus(100,10000,10)

unique words: 238
small_word: -59


In [49]:
rescale_corpus('corp.csv',-59)

In [27]:
bib=1

In [46]:
' '.join([1,2,3])

TypeError: sequence item 0: expected string, int found